<a href="https://colab.research.google.com/github/marinebon/HackingLimno2025/blob/main/03_Sample_environmental_data_at_each_occurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching Occurrence Data from pyobis



In [1]:
!pip install rasterio pandas geopandas matplotlib  requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 43.5 MB/s eta 0:00:00


Importing libraries

In [2]:
%matplotlib inline

mount google drive for saving result

In [3]:
from google.colab import drive
drive.mount('/content/drive')


import os
project_dir = '/content/drive/MyDrive/GSoC_SDM_Project'
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

Mounted at /content/drive


fetching occurrence data

Clean Occurrence Data Function

Data exploration


Function to prepare data for environmental sampling

In [ ]:
def prepare_for_environmental_sampling(
    df: pd.DataFrame,
    species_column: str = 'scientificName',
    lat_column: str = 'decimalLatitude',
    lon_column: str = 'decimalLongitude',
    additional_columns: Optional[List[str]] = None
) -> pd.DataFrame:

    columns = [species_column, lat_column, lon_column]

    if additional_columns:
        columns.extend([col for col in additional_columns if col in df.columns])

    prepared_df = df[columns].copy()

    prepared_df['occurrence_id'] = range(len(prepared_df))

    cols = ['occurrence_id'] + [col for col in prepared_df.columns if col != 'occurrence_id']
    prepared_df = prepared_df[cols]

    return prepared_df

This is just and example for testing the general functions


In [ ]:
# TODO: move this to pyOBIS notebok

def save_occurrence_data(
    df: pd.DataFrame,
    filename: str,
    format: str = 'csv'
) -> None:

    if format == 'csv':
        df.to_csv(f"{filename}.csv", index=False)
        print(f"Data saved to {filename}.csv")
    elif format == 'parquet':
        df.to_parquet(f"{filename}.parquet", index=False)
        print(f"Data saved to {filename}.parquet")
    else:
        raise ValueError(f"Unsupported format: {format}")


bringing in environmental data. Functions for Environmental Data Sampling

In [ ]:
!pip install rasterio
!pip install xarray
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.7 MB/s eta 0:00:00


In [ ]:
import rasterio
import xarray as xr
import numpy as np
from typing import Union, List, Tuple, Optional, Dict

In [ ]:
def add_environmental_data(
    df: pd.DataFrame,
    raster_path: str,
    column_name: str = 'temperature',
    lat_col: str = 'decimalLatitude',
    lon_col: str = 'decimalLongitude'
) -> pd.DataFrame:

    df_result = df.copy()

    with rasterio.open(raster_path) as src:
        coords = [(row[lon_col], row[lat_col]) for _, row in df.iterrows()]
        sampled_values = list(src.sample(coords))
        values = [val[0] if val[0] != src.nodata else np.nan for val in sampled_values]
        df_result[column_name] = values

    print(f"Added {column_name}: {df_result[column_name].notna().sum()}/{len(df_result)} valid values")
    return df_result

RASTER_PATH = f'{project_dir}/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc'

df_with_temp = add_environmental_data(df_clean, RASTER_PATH, 'temperature')

print(df_with_temp[['scientificName', 'decimalLatitude', 'decimalLongitude', 'eventDate', 'temperature']].head())

Added temperature: 12306/12374 valid values
  scientificName  decimalLatitude  decimalLongitude           eventDate  \
0      Mola mola         39.33100         -72.41040 2016-11-17 14:20:44   
3      Mola mola         40.16700         -70.03300 1981-08-02 11:14:00   
4      Mola mola         42.38549         -67.09194 2016-06-16 11:09:56   
6      Mola mola         39.73300         -71.53300 1979-07-18 11:44:00   
8      Mola mola         40.96870         -70.43540 2020-07-21 14:22:45   

   temperature  
0    34.304287  
3    33.463039  
4    32.325291  
6    34.602718  
8    32.050797  
